# *BehavePro*
---
Generates figures for entire folders of behaviour data

now in blind and unblind fashion

In [6]:
#import everything and set dpi here

import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100 #set dpi for viewing figs within the notebook
from tqdm import tqdm

### Load the infotable

In [2]:
directory = r"C:/Users/njana/Desktop/Behavior/Esr1ChronosStim/Day 4/" # include / at end
infofile = directory+"deets.csv"
infotable = pd.read_csv(infofile)
archive_name = "May1523_MeAeffStim"

infotable

,name,type,BlindID,MouseID,Type,Day,Trial,stimdur,Total,startf,endf,trunc end,ethofile
0,Tr1n04,standard,M4,BLIND,Chronos,4,1,30,37924,922,37924,0,Hardware-Nov1623_EsrStimD4-Trial 1-Arena 1
1,Tr2n02,standard,M2,BLIND,Chronos,4,2,30,38448,1341,38448,0,Hardware-Nov1623_EsrStimD4-Trial 2-Arena 1
2,Tr3n05,standard,M5,BLIND,Chronos,4,3,30,38160,1129,38160,0,Hardware-Nov1623_EsrStimD4-Trial 3-Arena 1
3,Tr4n03,standard,M3,BLIND,Chronos,4,4,30,38431,1051,38431,0,Hardware-Nov1623_EsrStimD4-Trial 4-Arena 1
4,Tr5n01,standard,M1,BLIND,Chronos,4,5,30,38440,807,38440,0,Hardware-Nov1623_EsrStimD4-Trial 5-Arena 1


### Archive data into ```hdf5``` for unblinding

In [6]:
''' archive data '''

behav_data = []
stim_data = []

for i in tqdm(range(infotable.shape[0])):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
        
    behav_data += [behavdata]
    stim_data += [stim]

archive = infotable
archive['behavior_traces'] = behav_data
archive['stimulus_traces'] = stim_data

archive.to_hdf(directory+archive_name+"_Blind_archive.h5", 'data' ,complevel = 0)

C:\Users\njana\AppData\Local\Temp\ipykernel_6184\1054319239.py:58: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['name', 'type', 'MouseID', 'Type', 'ethofile', 'behavior_traces',
       'stimulus_traces'],
      dtype='object')]

  archive.to_hdf(directory+archive_name+"_Blind_archive.h5", 'data' ,complevel = 0)


### Plot behavior traces with averages

In [16]:
''' Plot with Averages''' 

#for i in range(1):
for i in tqdm(range(infotable.shape[0])):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_Contact",	"Social_Contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_Move",	"Rest_Move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37)#, encoding='UTF-16le') # so stupid that this works but it does #if issue with UTF-8, use 16le

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
    
    BlindID = str(infotable.BlindID[i])
    MouseID = str(infotable.MouseID[i])
    Type = str(infotable.Type[i])        
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
    sns.set_style("ticks")

    #plt.subplots(9,2,gridspec_kw={'width_ratios': [time.shape[0],stimdur*fps*3]})
    plt.subplots(9,2,gridspec_kw={'width_ratios': [5,1]}) #15,1
    #plt.subplots_adjust(hspace=0)
    plt.subplots_adjust(wspace=0)


    plt.subplot(9,2,1)
    plt.plot(time,stim, color = 'blue')
    plt.ylabel("Stimulus", color = 'blue')
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    
    plt.subplot(9,2,2)
    plt.plot(avgtime,stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)], color = 'blue')
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    
    counter = 0
    plotnum = 3

    for k in range(8):
        
        plt.subplot(9,2,plotnum)
        plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[k], alpha = 0.3)
        counter += 1
        
        plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[k], alpha = 1)
        plt.ylabel(behavdata.columns[counter], color = colors[k])
        if plotnum < 17:
            plt.gca().set_xticks([])
        else:
            plt.gca().set_xticks(np.arange(min(time), max(time)+1, 300), labels = np.arange(min(time), max(time)+1, 300).astype(int))
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)
        plotnum += 1
        
        plt.subplot(9,2,plotnum)
        
        # calculate the average data
        behav = behavdata[behavdata.columns[counter]].to_numpy()
        window_behav = []
        for ele in stimstartidx:
            window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
        window_behav = np.array(window_behav)
        avg_behav = np.average(window_behav, axis = 0)
    
        # plot the average data
        plt.plot(avgtime,avg_behav, color = colors[k], alpha = 1)
        if max(avg_behav) == 0 or max(avg_behav) == 1:
            plt.ylim(0,1)
            plt.gca().set_yticks([1], labels = [1])
        else:
            plt.ylim(0,max(avg_behav)+0.1)
            plt.gca().set_yticks([max(avg_behav)+0.1], labels = [round(max(avg_behav)+0.1,1)])
            
        plt.gca().fill_between(avgtime, np.ones(2700),  where=stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)] > 0, facecolor='blue', alpha=.2)
        if plotnum < 17:
            plt.gca().set_xticks([])
        else:
            plt.gca().set_xticks([0,30,60,90], labels = [-30,0,30,60])
        
        sns.despine(bottom = True, left = True)        
            
        counter += 1   
        plotnum += 1
    
    plt.gcf().set_size_inches(16,8)

    if MouseID == 'BLIND':
        plt.gcf().suptitle("Resident "+BlindID+": Day "+day+" Trial "+trial, ha = 'left', x = 0.15, y = 0.92, fontsize = 16, fontweight = "bold")
        plt.savefig(directory+BlindID+"_D"+day+"Tr"+trial+".svg", dpi = 500)
        plt.savefig(directory+BlindID+"_D"+day+"Tr"+trial+".png", dpi = 500)
        plt.clf()
        
    else:
        plt.gcf().suptitle(Type+" Resident "+MouseID+" "+BlindID+": Day "+day+" Trial "+trial, ha = 'left', x = 0.15, y = 0.92, fontsize = 16, fontweight = "bold")
        plt.savefig(directory+Type+"_"+MouseID+"_"+BlindID+"D"+day+"Tr"+trial+".svg", dpi = 500)
        plt.savefig(directory+Type+"_"+MouseID+"_"+BlindID+"D"+day+"Tr"+trial+".png", dpi = 500)
        plt.clf()


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.41s/it]


<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>